In [ ]:
from fastai.vision import *

In [ ]:
import cv2

In [ ]:
path = './minds/image'

fnames = get_image_files(path)
lblnames = get_image_files('./minds/mask')

In [ ]:
img = open_image(fnames[0])
img

In [ ]:
lblnames[0]

In [ ]:
def get_y(x):
    #print('dbg0', type(x), x)
    #return x.replace('/image', '/mask')
    return 'minds/mask/%s%s' % (x.stem, x.suffix)

In [ ]:
def get_y_512(x):
    return 'dataset_20200708/mask/%s_mask%s' % (x.stem, x.suffix)

In [ ]:
class UnetSegmentationLabelList(SegmentationLabelList):
    def open(self, fn):
        return open_mask(fn, div=True)

In [ ]:
class UnetSegmentationItemList(SegmentationItemList):
    _label_cls, _square_show_res = UnetSegmentationLabelList, False

In [ ]:
data = (UnetSegmentationItemList
        .from_folder('minds/image')
        .split_by_rand_pct(0.2)
        .label_from_func(get_y, classes=['0', '1', '2', '3', '4'])
        .transform(get_transforms(), tfm_y=True, size = (128, 128))
        .databunch(bs=16, path='.')
        .normalize(imagenet_stats)
       )

In [ ]:
#dataset_20200708
data = (UnetSegmentationItemList
        .from_folder('dataset_20200708/image')
        .split_by_rand_pct(0.2)
        .label_from_func(get_y_512, classes=['0', '1'])
        .transform(get_transforms(), tfm_y=True)
        .databunch(bs=4, num_workers = 0)
        .normalize(imagenet_stats)
       )

In [ ]:
data.show_batch(3, figsize=(8,8), alpha=0.5)

In [ ]:
t = data.train_ds[1]

In [ ]:
t[0]

In [ ]:
data.train_dl

In [ ]:
xb, yb = next(iter(data.train_dl))

In [ ]:
xb.shape

In [ ]:
yb.shape

In [ ]:
t[1]

In [ ]:
t = data.valid_ds[0]

In [ ]:
t[1]

In [ ]:
t[0]

In [ ]:
learn= unet_learner(data, models.resnet34, self_attention=True, metrics=dice, wd=1e-2)

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
def dbg():
    import pdb; pdb.set_trace()
    data.show_batch()
dbg()

In [ ]:
# for i in range(101):
#     fn = 'minds/mask/%d.png' % i
#     img = cv2.imread(fn)
#     img = img * 60
#     cv2.imwrite(fn + '_enh.png', img)
#     plt.imshow(img)


In [ ]:
plt.imshow(cv2.imread('minds/mask/1.png_enh.png'))